In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from pathlib import Path
import h5py
import pandas as pd
import numpy as np
import csv
import os
import multiprocessing as mp
import patchify
import matplotlib.pyplot as plt
running_in= '/mnt/CPG'#'/data/pathology'#

## Creating train,tune, test splits 

In [9]:
witali_split=pd.read_csv(f'{running_in}/projects/Examode/wsi_experiments/colon/configs/colon_fold0.csv')
print(f'{len(witali_split)} WSI (catania+radboud):\n {witali_split.split.value_counts()}')
print(witali_split.head())
#Remove catania from the split
witali_split=witali_split[witali_split.category2!='catania']
witali_split.drop('classes',axis=1,inplace=True)
print(f'{len(witali_split)} WSI (just radboud):\n {witali_split.split.value_counts()}')
print(witali_split.head())

7019 WSI (catania+radboud):
 training      5468
validation    1403
testing        148
Name: split, dtype: int64
         pa    name category category2  normal  abnormal    split  \
0  17-10997  110849       ni   catania       1         0  testing   
1  17-15278  128253      hgd   catania       0         1  testing   
2  17-15545  129312      lgd   catania       0         1  testing   
3  17-19372  143950   cancer   catania       0         1  testing   
4  17-19392  144484   cancer   catania       0         1  testing   

            classes  ni  hyperplastic  lgd  hgd  cancer  
0                ni   1             0    0    0       0  
1           lgd hgd   0             0    1    1       0  
2  hyperplastic lgd   0             1    1    0       0  
3            cancer   0             0    0    0       1  
4            cancer   0             0    0    0       1  
6943 WSI (just radboud):
 training      5468
validation    1403
testing         72
Name: split, dtype: int64
                

In [28]:
#count the category of each image
print('Normal class: '+ str(witali_split.where(witali_split.normal==1).dropna()['category'].count()))
print(witali_split.where(witali_split.normal==1).dropna()['category'].value_counts())
print('Abnormal class: '+ str(witali_split.where(witali_split.abnormal==1).dropna()['category'].count()))
print(witali_split.where(witali_split.abnormal==1).dropna()['category'].value_counts())
print('-'*50)
print('Splits:')
print(witali_split.split.value_counts())
print('Training'+'-'*20)
print(witali_split.where(witali_split.split=='training').dropna()['category'].value_counts())
print('Validation'+'-'*20)
print(witali_split.where(witali_split.split=='validation').dropna()['category'].value_counts())
print('Testing'+'-'*20)
print(witali_split.where(witali_split.split=='testing').dropna()['category'].value_counts())

Normal class: 4945
ni              4098
hyperplastic     847
Name: category, dtype: int64
Abnormal class: 1998
lgd       1501
cancer     296
hgd        201
Name: category, dtype: int64
--------------------------------------------------
Splits:
training      5468
validation    1403
testing         72
Name: split, dtype: int64
Training--------------------
ni              3260
lgd             1189
hyperplastic     651
cancer           222
hgd              146
Name: category, dtype: int64
Validation--------------------
ni              824
lgd             292
hyperplastic    184
cancer           60
hgd              43
Name: category, dtype: int64
Testing--------------------
lgd             20
cancer          14
ni              14
hgd             12
hyperplastic    12
Name: category, dtype: int64


Lets create the csv with the training,validation and testing sets.

In [20]:
processed_wsi=pd.read_csv(f'{running_in}/projects/pathology-self-supervision/data/examode_colon/processed_wsi2region_h5.csv')
#get a df from witali split with the wsi contained in processed_wsi
real_split=witali_split[witali_split.name.isin(processed_wsi.WSI_name)]
split={'training':'train', 'validation':'tune', 'testing':'test'}
saving_dir=f'{running_in}/projects/pathology-self-supervision/data/examode_colon/WSI_experiments'
for k,v in split.items():
    with open(f'{saving_dir}/{v}.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(['slide_id','label'])
        for index, row in real_split.where(real_split.split==k).dropna().iterrows():
            writer.writerow([row['name'], int(row['abnormal'])])

In [19]:
witali_names=witali_split.name.to_list()
processed_names=processed_wsi.WSI_name.to_list()
missing_names_2=[name for name in processed_names if name not in witali_names]
missing_names=[name for name in witali_names if name not in processed_names]
print(f'Missing to extract regions from {len(missing_names)} images that are in the witali split')
print(f'There are {len(missing_names_2)} images in my pretraining that are not in Witalis split')
#Get the a dataframe with the missing names from witali split dataframe
missing_df=witali_split[witali_split.name.isin(missing_names)]

Missing to extract regions from 434 images that are in the witali split
There are 54 images in my pretraining that are not in Witalis split


In [23]:
extra_images=processed_wsi[processed_wsi.WSI_name.isin(missing_names_2)]
extra_images.Label.value_counts()

colon_non_cancer_wo_d    39
colon_cancer_wo_d         7
colon_non_cancer_lgd      5
colon_non_cancer_hgd      3
Name: Label, dtype: int64

In [15]:
print('Missing Normal class:')
print(missing_df.where(missing_df.normal == 1).dropna()['category'].value_counts())
print('Missing Abnormal class:')
print(missing_df.where(missing_df.abnormal == 1).dropna()['category'].value_counts())
print('-'*30)
print('Missing Splits:')
print(missing_df.split.value_counts())
print('Missing Training'+'-'*20)
print(missing_df.where(missing_df.split == 'training').dropna()['category'].value_counts() if len(missing_df.where(
    missing_df.split == 'training').dropna()['category'].value_counts()) > 0 else 'No missing training images')
print('Missing Validation'+'-'*20)
print(missing_df.where(missing_df.split == 'validation').dropna()['category'].value_counts()if len(missing_df.where(
    missing_df.split == 'validation').dropna()['category'].value_counts()) > 0 else 'No missing validation images')
print('Missing Testing'+'-'*20)
print(missing_df.where(missing_df.split == 'testing').dropna()['category'].value_counts() if len(missing_df.where(
    missing_df.split == 'testing').dropna()['category'].value_counts()) > 0 else 'No missing testing images')


Missing Normal class:
ni    432
Name: category, dtype: int64
Missing Abnormal class:
lgd    2
Name: category, dtype: int64
------------------------------
Missing Splits:
training      352
validation     82
Name: split, dtype: int64
Missing Training--------------------
ni     351
lgd      1
Name: category, dtype: int64
Missing Validation--------------------
ni     81
lgd     1
Name: category, dtype: int64
Missing Testing--------------------
No missing testing images


From this we can say that I am pretraining with 54 wholeslide images not in Witali split, and that I will train the last block from HIPT with 434 unseen wholeslide images during pretraining. 

I have the whole test set (72 images) processed.

From those 434 missing images to process, 432 (351 in train/81 in validation) are from ni category (class normal) which is the category with more samples (3260 in training/824 in validation for a total of 4084).The other 2 (1 in train/1 in validation) images are from the lgd category (class abnormal) which is the category of abnormal with more samples (1189 in training/292 in validation for a total of 1481)

In [22]:
wsi_path = list(Path(
    f"{running_in}/archives/gastrointestinal/examode_biopsies_radboudumc_colon/packed_blocks_sp05/packed").rglob("*.tif"))
wsi_names = [path.stem for path in wsi_path]
missing_2_get_regions = [name for name in missing_names if name in wsi_names]
wsi_in_neither=[name for name in wsi_names if name not in processed_names and name not in witali_names] #WSI images that are in Blissey but not in Witalis split nor in my pretraining(not in the ggt excel)

,pa,name,category,category2,normal,abnormal,split,ni,hyperplastic,lgd,hgd,cancer
1802,EX_S03_P001118,EX_S03_P001118_C0001_B1,ni,rumc,1,0,validation,1,0,0,0,0
1994,EX_S03_P001230,EX_S03_P001230_C0001_B1,ni,rumc,1,0,training,1,0,0,0,0
2078,EX_S03_P001268,EX_S03_P001268_C0001_B2,lgd,rumc,0,1,validation,0,0,1,0,0
5605,EX_S03_P003261,EX_S03_P003261_C0001_B3,lgd,rumc,0,1,training,0,0,1,0,0
6237,EX_S03_P003690,EX_S03_P003690_C0001_B1,ni,rumc,1,0,training,1,0,0,0,0
6588,EX_S03_P003899,EX_S03_P003899_C0001_B2,ni,rumc,1,0,training,1,0,0,0,0
6606,EX_S03_P003909,EX_S03_P003909_C0001_B2,ni,rumc,1,0,training,1,0,0,0,0


However the 427/434 images missing are not in Blissey. Five images from the wsi in blissey are from normal class (4 train/1 val) and two from abnormal class (1 train/1 val). 

So after processing those, I would be left with 427 (347 in train/80 in validation) from ni category (class normal) missing.

In [23]:
#Do a csv file with the missing names that are not in blissey but in Witalis split (I would like to extract regions from them to make the training results comparable with Witalis results)
missing_df[~missing_df.name.isin(missing_2_get_regions)].to_csv(f'{saving_dir}/missing_in_blissey.csv',index=False)

In [27]:
#Do a csv file with the missing images paths
wsi_2_be_processed=[wsi_p for wsi_p in wsi_path if wsi_p.stem in missing_2_get_regions]
# replace /mnt/CPG with /data/pathology
wsi_2_be_processed=[str(wsi_p).replace('/mnt/CPG','/data/pathology') for wsi_p in wsi_2_be_processed]
mask_path =list(Path(f"{running_in}/archives/gastrointestinal/examode_biopsies_radboudumc_colon/packed_blocks_sp05/tissue_masks").rglob("*.tif"))
#get the mask path for the missing images
mask_2_be_processed=[mask_p for mask_p in mask_path if ("_").join(mask_p.stem.split("_")[:-1]) in missing_2_get_regions]
# replace /mnt/CPG with /data/pathology
mask_2_be_processed=[str(mask_p).replace('/mnt/CPG','/data/pathology') for mask_p in mask_2_be_processed]
missing_df[missing_df.name.isin(missing_2_get_regions)] #Images that are in Blissey and in Witalis split but not in my pretraining, I will get the regions and extract features from them to train the last HIPT layer

,pa,name,category,category2,normal,abnormal,split,ni,hyperplastic,lgd,hgd,cancer
1802,EX_S03_P001118,EX_S03_P001118_C0001_B1,ni,rumc,1,0,validation,1,0,0,0,0
1994,EX_S03_P001230,EX_S03_P001230_C0001_B1,ni,rumc,1,0,training,1,0,0,0,0
2078,EX_S03_P001268,EX_S03_P001268_C0001_B2,lgd,rumc,0,1,validation,0,0,1,0,0
5605,EX_S03_P003261,EX_S03_P003261_C0001_B3,lgd,rumc,0,1,training,0,0,1,0,0
6237,EX_S03_P003690,EX_S03_P003690_C0001_B1,ni,rumc,1,0,training,1,0,0,0,0
6588,EX_S03_P003899,EX_S03_P003899_C0001_B2,ni,rumc,1,0,training,1,0,0,0,0
6606,EX_S03_P003909,EX_S03_P003909_C0001_B2,ni,rumc,1,0,training,1,0,0,0,0


In [28]:
mask_2_be_processed.sort()
wsi_2_be_processed.sort()
#Create a csv file with the missing images paths
csv_dir=f'{running_in}/projects/pathology-self-supervision/H2SP_csv/colon_not_in_pretraining.csv'
with open(csv_dir, 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['slide_path','mask_path'])
    writer.writerows(zip(wsi_2_be_processed,mask_2_be_processed))

/data/pathology/archives/gastrointestinal/examode_biopsies_radboudumc_colon/packed_blocks_sp05/packed/EX_S03_P001268_C0001_B2.tif is just pixelated stuff
/data/pathology/archives/gastrointestinal/examode_biopsies_radboudumc_colon/packed_blocks_sp05/packed/EX_S03_P003261_C0001_B3.tif is just pixelated stuff
/data/pathology/archives/gastrointestinal/examode_biopsies_radboudumc_colon/packed_blocks_sp05/packed/EX_S03_P003690_C0001_B1.tif is just pixelated stuff
/data/pathology/archives/gastrointestinal/examode_biopsies_radboudumc_colon/packed_blocks_sp05/packed/EX_S03_P003899_C0001_B2.tif is just pixelated stuff
/data/pathology/archives/gastrointestinal/examode_biopsies_radboudumc_colon/packed_blocks_sp05/packed/EX_S03_P003909_C0001_B2.tif is just pixelated stuff

Are they different in Snellius?


## Dividing the process list of feature extraction to process it in parallel

In [2]:
processed_feature=pd.read_csv(f'{running_in}/projects/pathology-self-supervision/data/examode_colon/WSI_experiments/4096_HIPT_TCGA_ckpt/features/process_list_global_pretraining.csv')
print(processed_feature.where(processed_feature.process==0).dropna()['process'].count())
mask = processed_feature["process"] == 1
process_stack = processed_feature[mask]
total = len(process_stack)
already_processed = len(processed_feature) - total
print(f'Missing to process {total}')
print(f'Already processed {already_processed}')
#take half of the process stack 
process_stack_2 = process_stack[-int(total/2):].index.tolist()


6563
Missing to process 0
Already processed 6563
